In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
import pandas as pd
import numpy as np
import cv2
from pathlib import Path
import matplotlib.pyplot as plt

print("✅ Torch:", torch.__version__)
print("CUDA доступен:", torch.cuda.is_available())

✅ Torch: 2.9.0+cpu
CUDA доступен: False


In [9]:
# Пути
PROJECT_ROOT = Path(r"E:\work\mvp_ultrasound")
CSV_PATH = PROJECT_ROOT / "data" / "processed" / "dataset_index.csv"

# Загружаем CSV
df = pd.read_csv(CSV_PATH)

print("Всего строк:", len(df))
print("\nПервые 5 строк:")
print(df.head())

print("\nУникальные классы:", df['label'].unique())
print("Разбиения (train/val/test):")
print(df['split'].value_counts())

Всего строк: 359

Первые 5 строк:
   split                                               path   label
0  train  E:\work\mvp_ultrasound\data\processed\train\be...  benign
1  train  E:\work\mvp_ultrasound\data\processed\train\be...  benign
2  train  E:\work\mvp_ultrasound\data\processed\train\be...  benign
3  train  E:\work\mvp_ultrasound\data\processed\train\be...  benign
4  train  E:\work\mvp_ultrasound\data\processed\train\be...  benign

Уникальные классы: ['benign' 'malignant' 'normal']
Разбиения (train/val/test):
split
train         126
test          123
validation    110
Name: count, dtype: int64


In [10]:
from PIL import Image

# Словарь для кодирования меток
label2id = {"benign": 0, "malignant": 1, "normal": 2}
id2label = {v: k for k, v in label2id.items()}

class UltrasoundDataset(Dataset):
    def _init_(self, df, transform=None):
        self.df = df.reset_index(drop=True)
        self.transform = transform

    def _len_(self):
        return len(self.df)

    def _getitem_(self, idx):
        row = self.df.iloc[idx]
        img_path = row["path"]
        label = label2id[row["label"]]
        
        # Загружаем изображение
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (256, 256))
        
        # Преобразуем в PIL для совместимости с torchvision.transforms
        img = Image.fromarray(img)
        
        if self.transform:
            img = self.transform(img)
        else:
            img = transforms.ToTensor()(img)
        
        return img, torch.tensor(label, dtype=torch.long)

In [11]:
# Разделим данные
df_train = df[df["split"] == "train"]
df_val = df[df["split"] == "validation"]
df_test = df[df["split"] == "test"]

# Преобразования (аугментации)
train_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor()
])

val_transforms = transforms.Compose([
    transforms.ToTensor()
])

# Создаём датасеты
train_dataset = UltrasoundDataset(df_train, transform=train_transforms)
val_dataset = UltrasoundDataset(df_val, transform=val_transforms)
test_dataset = UltrasoundDataset(df_test, transform=val_transforms)

# DataLoader’ы
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

print("✅ DataLoaders созданы")
print("Размеры:")
print("train:", len(train_dataset), "примеров")
print("validation:", len(val_dataset), "примеров")
print("test:", len(test_dataset), "примеров")

TypeError: UltrasoundDataset() takes no arguments

In [12]:
import inspect

print("Объект:", UltrasoundDataset)
print("Тип:", type(UltrasoundDataset))

# Попробуем узнать сигнатуру _init_
try:
    print("Сигнатура _init:", inspect.signature(UltrasoundDataset.init_))
except Exception as e:
    print("Не удалось получить сигнатуру:", e)`

SyntaxError: invalid syntax (591619184.py, line 10)

In [13]:
`import inspect

print("Объект:", UltrasoundDataset)
print("Тип:", type(UltrasoundDataset))

# Попробуем узнать сигнатуру _init_
try:
    print("Сигнатура _init:", inspect.signature(UltrasoundDataset.init_))
except Exception as e:
    print("Не удалось получить сигнатуру:", e)

SyntaxError: invalid syntax (3970848764.py, line 1)

In [14]:
import inspect

print("Объект:", UltrasoundDataset)
print("Тип:", type(UltrasoundDataset))

# Попробуем узнать сигнатуру _init_
try:
    print("Сигнатура _init:", inspect.signature(UltrasoundDataset.init_))
except Exception as e:
    print("Не удалось получить сигнатуру:", e)

Объект: <class '__main__.UltrasoundDataset'>
Тип: <class 'type'>
Не удалось получить сигнатуру: type object 'UltrasoundDataset' has no attribute 'init_'


In [15]:
from PIL import Image
from torch.utils.data import Dataset
import torchvision.transforms as transforms
import cv2
import torch

label2id = {"benign": 0, "malignant": 1, "normal": 2}
id2label = {v: k for k, v in label2id.items()}

class UltrasoundDatasetV2(Dataset):
    def _init_(self, df, transform=None):
        self.df = df.reset_index(drop=True)
        self.transform = transform

    def _len_(self):
        return len(self.df)

    def _getitem_(self, idx):
        row = self.df.iloc[idx]
        img_path = row["path"]
        label = label2id[row["label"]]

        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (256, 256))

        img = Image.fromarray(img)
        if self.transform:
            img = self.transform(img)
        else:
            img = transforms.ToTensor()(img)

        return img, torch.tensor(label, dtype=torch.long)

In [16]:
from torch.utils.data import DataLoader
import torchvision.transforms as transforms

# Разделяем данные
df_train = df[df["split"] == "train"]
df_val = df[df["split"] == "validation"]
df_test = df[df["split"] == "test"]

# Преобразования (аугментации)
train_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor()
])

val_transforms = transforms.Compose([
    transforms.ToTensor()
])

# Создаём датасеты с НОВЫМ классом
train_dataset = UltrasoundDatasetV2(df_train, transform=train_transforms)
val_dataset = UltrasoundDatasetV2(df_val, transform=val_transforms)
test_dataset = UltrasoundDatasetV2(df_test, transform=val_transforms)

# DataLoader’ы
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

print("✅ DataLoaders созданы")
print("train:", len(train_dataset), "примеров")
print("validation:", len(val_dataset), "примеров")
print("test:", len(test_dataset), "примеров")

TypeError: UltrasoundDatasetV2() takes no arguments

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
import pandas as pd
import numpy as np
import cv2
from pathlib import Path
import matplotlib.pyplot as plt
from PIL import Image

print("✅ Torch:", torch.__version__)
print("CUDA доступен:", torch.cuda.is_available())

✅ Torch: 2.9.0+cpu
CUDA доступен: False


In [2]:
PROJECT_ROOT = Path(r"E:\work\mvp_ultrasound")
CSV_PATH = PROJECT_ROOT / "data" / "processed" / "dataset_index.csv"
df = pd.read_csv(CSV_PATH)
print("Всего строк:", len(df))

Всего строк: 359


In [3]:
label2id = {"benign": 0, "malignant": 1, "normal": 2}
id2label = {v: k for k, v in label2id.items()}

class UltrasoundDatasetFresh(Dataset):
    def __init__(self, df, transform=None):
        self.df = df.reset_index(drop=True)
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = row["path"]
        label = label2id[row["label"]]

        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (256, 256))

        img = Image.fromarray(img)
        if self.transform:
            img = self.transform(img)
        else:
            img = transforms.ToTensor()(img)

        return img, torch.tensor(label, dtype=torch.long)

In [4]:
df_train = df[df["split"] == "train"]
df_val = df[df["split"] == "validation"]
df_test = df[df["split"] == "test"]

train_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor()
])

val_transforms = transforms.Compose([
    transforms.ToTensor()
])

train_dataset = UltrasoundDatasetFresh(df_train, transform=train_transforms)
val_dataset = UltrasoundDatasetFresh(df_val, transform=val_transforms)
test_dataset = UltrasoundDatasetFresh(df_test, transform=val_transforms)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

print("✅ DataLoaders созданы")
print("train:", len(train_dataset), "примеров")
print("validation:", len(val_dataset), "примеров")
print("test:", len(test_dataset), "примеров")

✅ DataLoaders созданы
train: 126 примеров
validation: 110 примеров
test: 123 примеров


In [5]:
import torch.nn.functional as F

# --- Базовая U-Net энкодер-декодер ---
class UNetBlock(nn.Module):
    def _init_(self, in_ch, out_ch):
        super()._init_()
        self.conv = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, 3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_ch, out_ch, 3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.conv(x)


class HybridUNetClassifier(nn.Module):
    def _init_(self, num_classes=3):
        super()._init_()
        # U-Net encoder
        self.enc1 = UNetBlock(1, 32)
        self.enc2 = UNetBlock(32, 64)
        self.enc3 = UNetBlock(64, 128)

        # Down-sampling
        self.pool = nn.MaxPool2d(2, 2)

        # Decoder
        self.up1 = nn.ConvTranspose2d(128, 64, 2, stride=2)
        self.dec1 = UNetBlock(128, 64)
        self.up2 = nn.ConvTranspose2d(64, 32, 2, stride=2)
        self.dec2 = UNetBlock(64, 32)

        # Output segmentation map
        self.seg_head = nn.Conv2d(32, 1, kernel_size=1)

        # Classification head
        self.classifier = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Flatten(),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        # Encoder
        x1 = self.enc1(x)
        x2 = self.enc2(self.pool(x1))
        x3 = self.enc3(self.pool(x2))

        # Decoder (for segmentation)
        x_up1 = self.up1(x3)
        x_cat1 = torch.cat([x_up1, x2], dim=1)
        x_dec1 = self.dec1(x_cat1)
        x_up2 = self.up2(x_dec1)
        x_cat2 = torch.cat([x_up2, x1], dim=1)
        x_dec2 = self.dec2(x_cat2)

        seg_output = torch.sigmoid(self.seg_head(x_dec2))  # segmentation output

        # Classification (from the deepest encoder feature)
        cls_logits = self.classifier(x3)

        return seg_output, cls_logits

In [6]:
device = torch.device("cpu")  # у нас CPU
model = HybridUNetClassifier(num_classes=3).to(device)

# возьмём один батч из train_loader
xb, yb = next(iter(train_loader))  # xb: [B, 1, 256, 256], yb: [B]
xb, yb = xb.to(device), yb.to(device)

seg_out, cls_logits = model(xb)
print("Форма входа:", xb.shape)
print("Сегментация (seg_out):", seg_out.shape, "диапазон:", (seg_out.min().item(), seg_out.max().item()))
print("Классификация (cls_logits):", cls_logits.shape)

TypeError: HybridUNetClassifier.__init__() got an unexpected keyword argument 'num_classes'

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class UNetBlockV2(nn.Module):
    def _init_(self, in_ch, out_ch):
        super()._init_()
        self.conv = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, 3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_ch, out_ch, 3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.conv(x)


class HybridUNetClassifierV2(nn.Module):
    def _init_(self, num_classes=3):
        super()._init_()
        # Encoder
        self.enc1 = UNetBlockV2(1, 32)
        self.enc2 = UNetBlockV2(32, 64)
        self.enc3 = UNetBlockV2(64, 128)
        self.pool = nn.MaxPool2d(2, 2)

        # Decoder
        self.up1 = nn.ConvTranspose2d(128, 64, 2, stride=2)
        self.dec1 = UNetBlockV2(128, 64)
        self.up2 = nn.ConvTranspose2d(64, 32, 2, stride=2)
        self.dec2 = UNetBlockV2(64, 32)

        # Heads
        self.seg_head = nn.Conv2d(32, 1, kernel_size=1)
        self.classifier = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Flatten(),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        # Encoder
        x1 = self.enc1(x)
        x2 = self.enc2(self.pool(x1))
        x3 = self.enc3(self.pool(x2))

        # Decoder (for segmentation)
        x_up1 = self.up1(x3)
        x_cat1 = torch.cat([x_up1, x2], dim=1)
        x_dec1 = self.dec1(x_cat1)
        x_up2 = self.up2(x_dec1)
        x_cat2 = torch.cat([x_up2, x1], dim=1)
        x_dec2 = self.dec2(x_cat2)

        seg_output = torch.sigmoid(self.seg_head(x_dec2))
        cls_logits = self.classifier(x3)

        return seg_output, cls_logits

In [3]:
device = torch.device("cpu")
model = HybridUNetClassifierV2(num_classes=3).to(device)

xb, yb = next(iter(train_loader))
xb, yb = xb.to(device), yb.to(device)

seg_out, cls_logits = model(xb)

print("Форма входа:", xb.shape)
print("Сегментация (seg_out):", seg_out.shape)
print("Классификация (cls_logits):", cls_logits.shape)

NameError: name 'HybridUNetClassifierV2' is not defined

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
import pandas as pd
import numpy as np
import cv2
from PIL import Image
from pathlib import Path
import matplotlib.pyplot as plt

print("✅ Torch:", torch.__version__)
print("CUDA доступен:", torch.cuda.is_available())

✅ Torch: 2.9.0+cpu
CUDA доступен: False


In [2]:
PROJECT_ROOT = Path(r"E:\work\mvp_ultrasound")
CSV_PATH = PROJECT_ROOT / "data" / "processed" / "dataset_index.csv"
df = pd.read_csv(CSV_PATH)
print("Всего строк:", len(df))

Всего строк: 359


In [11]:
label2id = {"benign": 0, "malignant": 1, "normal": 2}
id2label = {v: k for k, v in label2id.items()}

class UltrasoundDatasetFresh(Dataset):
    def __init__(self, df, transform=None):
        self.df = df.reset_index(drop=True)
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img = cv2.imread(row["path"], cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (256, 256))
        img = Image.fromarray(img)
        img = self.transform(img) if self.transform else transforms.ToTensor()(img)
        label = torch.tensor(label2id[row["label"]], dtype=torch.long)
        return img, label

In [10]:
df_train = df[df["split"] == "train"]
df_val = df[df["split"] == "validation"]
df_test = df[df["split"] == "test"]

train_tf = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor()
])
val_tf = transforms.Compose([transforms.ToTensor()])

train_loader = DataLoader(UltrasoundDatasetFresh(df_train, train_tf), batch_size=8, shuffle=True)
val_loader   = DataLoader(UltrasoundDatasetFresh(df_val, val_tf),   batch_size=8, shuffle=False)
test_loader  = DataLoader(UltrasoundDatasetFresh(df_test, val_tf),  batch_size=8, shuffle=False)

print("✅ DataLoaders готовы")

NameError: name 'df' is not defined

In [8]:
class UNetBlockNew(nn.Module):
    def __init__(self, in_ch, out_ch):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, 3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(),
            nn.Conv2d(out_ch, out_ch, 3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU()
        )

    def forward(self, x): return self.conv(x)

class HybridUNetClassifierClean(nn.Module):
    def __init__(self, num_classes=3):
        super().__init__()
        self.enc1 = UNetBlockNew(1, 32)
        self.enc2 = UNetBlockNew(32, 64)
        self.enc3 = UNetBlockNew(64, 128)
        self.pool = nn.MaxPool2d(2)
        self.up1  = nn.ConvTranspose2d(128, 64, 2, stride=2)
        self.dec1 = UNetBlockNew(128, 64)
        self.up2  = nn.ConvTranspose2d(64, 32, 2, stride=2)
        self.dec2 = UNetBlockNew(64, 32)
        self.seg_head = nn.Conv2d(32, 1, 1)
        self.classifier = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Flatten(),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        x1 = self.enc1(x)
        x2 = self.enc2(self.pool(x1))
        x3 = self.enc3(self.pool(x2))
        x_up1 = self.up1(x3)
        x_dec1 = self.dec1(torch.cat([x_up1, x2], 1))
        x_up2 = self.up2(x_dec1)
        x_dec2 = self.dec2(torch.cat([x_up2, x1], 1))
        seg = torch.sigmoid(self.seg_head(x_dec2))
        cls = self.classifier(x3)
        return seg, cls

In [9]:
device = torch.device("cpu")
model = HybridUNetClassifierClean(num_classes=3).to(device)
xb, yb = next(iter(train_loader))
seg, cls = model(xb)
print("seg:", seg.shape, "cls:", cls.shape)

NameError: name 'train_loader' is not defined

In [7]:
# Лосс для классификации (CrossEntropy)
cls_criterion = nn.CrossEntropyLoss()

# Пока сегментацию не обучаем (нет масок)
# Но можно считать seg_loss = 0, чтобы модель не ломалась при комбинировании
def total_loss(cls_logits, y_true):
    cls_loss = cls_criterion(cls_logits, y_true)
    return cls_loss  # пока только классификация

# Оптимизатор
optimizer = optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)

print("✅ Оптимизатор и лосс готовы")

✅ Оптимизатор и лосс готовы


In [8]:
def accuracy(logits, y_true):
    preds = torch.argmax(logits, dim=1)
    return (preds == y_true).float().mean().item()

@torch.no_grad()
def evaluate(model, loader, device="cpu"):
    model.eval()
    total_loss, total_acc, total_n = 0.0, 0.0, 0
    for xb, yb in loader:
        xb, yb = xb.to(device), yb.to(device)
        seg_out, cls_logits = model(xb)
        loss = cls_criterion(cls_logits, yb)
        acc = accuracy(cls_logits, yb)
        bs = yb.size(0)
        total_loss += loss.item() * bs
        total_acc += acc * bs
        total_n += bs
    return total_loss / total_n, total_acc / total_n

print("✅ Функции evaluate() и accuracy() готовы")

✅ Функции evaluate() и accuracy() готовы


In [9]:
EPOCHS = 3

for epoch in range(1, EPOCHS + 1):
    model.train()
    running_loss, running_acc, seen = 0.0, 0.0, 0

    for xb, yb in train_loader:
        xb, yb = xb.to(device), yb.to(device)

        optimizer.zero_grad()
        seg_out, cls_logits = model(xb)
        loss = total_loss(cls_logits, yb)
        loss.backward()
        optimizer.step()

        acc = accuracy(cls_logits, yb)
        bs = yb.size(0)
        running_loss += loss.item() * bs
        running_acc += acc * bs
        seen += bs

    train_loss = running_loss / seen
    train_acc = running_acc / seen
    val_loss, val_acc = evaluate(model, val_loader, device)

    print(f"[{epoch}/{EPOCHS}] "
          f"train_loss={train_loss:.4f}, acc={train_acc:.3f} | "
          f"val_loss={val_loss:.4f}, acc={val_acc:.3f}")

[1/3] train_loss=1.1266, acc=0.341 | val_loss=1.0714, acc=0.373
[2/3] train_loss=1.0258, acc=0.381 | val_loss=1.5027, acc=0.391
[3/3] train_loss=1.0068, acc=0.373 | val_loss=1.4176, acc=0.391


In [10]:
from torchvision import transforms

# Среднее и стандартное отклонение для grayscale-изображений
mean_gray = [0.5]
std_gray = [0.5]

train_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.RandomResizedCrop(256, scale=(0.8, 1.0)),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean_gray, std=std_gray)
])

val_transforms = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean_gray, std=std_gray)
])

train_loader = DataLoader(UltrasoundDatasetFresh(df_train, transform=train_transforms), batch_size=8, shuffle=True)
val_loader = DataLoader(UltrasoundDatasetFresh(df_val, transform=val_transforms), batch_size=8, shuffle=False)

print("✅ Трансформации и нормализация обновлены")

✅ Трансформации и нормализация обновлены


In [11]:
from collections import Counter
import torch

# Подсчитаем количество примеров каждого класса
class_counts = Counter(df_train["label"])
print("📊 Количество примеров в train:", class_counts)

# Преобразуем в веса (меньше примеров = больший вес)
total = sum(class_counts.values())
class_weights = [total / class_counts[l] for l in ["benign", "malignant", "normal"]]
class_weights = torch.tensor(class_weights, dtype=torch.float32)

print("⚖️ Веса классов:", class_weights)

# Новый лосс с учетом дисбаланса
cls_criterion = nn.CrossEntropyLoss(weight=class_weights)

print("✅ Балансировка классов добавлена в лосс")

📊 Количество примеров в train: Counter({'benign': 50, 'normal': 50, 'malignant': 26})
⚖️ Веса классов: tensor([2.5200, 4.8462, 2.5200])
✅ Балансировка классов добавлена в лосс


In [12]:
from torch.optim.lr_scheduler import ReduceLROnPlateau

optimizer = optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)

# Scheduler: уменьшает LR в 2 раза, если 2 эпохи подряд нет улучшения валидации
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2, verbose=True)

print("✅ Scheduler добавлен (ReduceLROnPlateau)")

TypeError: ReduceLROnPlateau.__init__() got an unexpected keyword argument 'verbose'

In [13]:
from torch.optim.lr_scheduler import ReduceLROnPlateau

optimizer = optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)

# Упрощённый вариант без verbose
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2)

print("✅ Scheduler добавлен (ReduceLROnPlateau, совместимая версия)")

✅ Scheduler добавлен (ReduceLROnPlateau, совместимая версия)


In [14]:
EPOCHS = 5

for epoch in range(1, EPOCHS + 1):
    model.train()
    running_loss, running_acc, seen = 0.0, 0.0, 0

    for xb, yb in train_loader:
        xb, yb = xb.to(device), yb.to(device)

        optimizer.zero_grad()
        seg_out, cls_logits = model(xb)
        loss = total_loss(cls_logits, yb)
        loss.backward()
        optimizer.step()

        acc = accuracy(cls_logits, yb)
        bs = yb.size(0)
        running_loss += loss.item() * bs
        running_acc += acc * bs
        seen += bs

    train_loss = running_loss / seen
    train_acc = running_acc / seen
    val_loss, val_acc = evaluate(model, val_loader, device)

    # обновляем scheduler
    scheduler.step(val_loss)

    print(f"[{epoch}/{EPOCHS}] "
          f"train_loss={train_loss:.4f}, acc={train_acc:.3f} | "
          f"val_loss={val_loss:.4f}, acc={val_acc:.3f} | "
          f"lr={optimizer.param_groups[0]['lr']:.6f}")

[1/5] train_loss=1.0273, acc=0.373 | val_loss=1.0966, acc=0.418 | lr=0.001000
[2/5] train_loss=0.9628, acc=0.437 | val_loss=1.0389, acc=0.491 | lr=0.001000
[3/5] train_loss=0.9599, acc=0.373 | val_loss=1.0888, acc=0.482 | lr=0.001000
[4/5] train_loss=0.9470, acc=0.500 | val_loss=1.1519, acc=0.373 | lr=0.001000
[5/5] train_loss=0.9615, acc=0.444 | val_loss=1.3215, acc=0.245 | lr=0.000500


In [12]:
from pathlib import Path
import torch

models_dir = Path(r"E:\work\mvp_ultrasound\models")
models_dir.mkdir(exist_ok=True)

save_path = models_dir / "mvp_model.pt"
torch.save(model.state_dict(), save_path)

print(f"✅ Модель сохранена по пути: {save_path}")

✅ Модель сохранена по пути: E:\work\mvp_ultrasound\models\mvp_model.pt
